### Setup Libraries

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSD
import prody as prd # really annoying to install. Have to put directly into library folder.
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from numpy import * 
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


#### Set the run types being observed, # of runs for each run type, and RPI
Terminology:
- Run Type: what the MD simulation is modeling. Ex: wild type protein, res109 mutated to W, ect.
- Runs: which run of a run type, as each run type would be simulated 3+ times to improve data accuracy.

Folder Setup:
- Each run type should be stored in a folder that lines up with the name put in run_types list.
- Then each run_type folder should have another set of folders that hold each different run. Each folder should have the same name that was put into run_count list.

In [ ]:
run_types = ['WT_v11','S440W_v1'] # add the name of each 
run_count = ['Run1','Run2','Run3'] # add the name of each 
frames = ["{:04}".format(i) for i in range(1,401)] # change last number based on number of frames + 1
sim_length = 200 #how long the simulation is in nanoseconds

#### Find Distance Between Pocket and Ligand
*Needs output from VMD distances script*

In [ ]:
# import the VMD distance's found
pocket_distances = {}
for run_type in run_types:
    pocket_distances[run_type] = []
    for run in range(len(run_count)): 
        pocket_distances[run_type].append([])
        with open(f"data_storage/WT_S440W/FAD_pocket_distance/posedistance_{run_type.split("_")[0]}{run+1}.dat", "r") as f:
            text = f.read()
            text = text.split("\n")
            for line in text:
                split_line = line.split("	")
                if len(split_line) > 1:
                    pocket_distances[run_type][-1].append(float(line.split("	")[1]))


    # average it all together
    pocket_distances[run_type] = np.swapaxes(pocket_distances[run_type],0, 1)
    pocket_distances[run_type] = [np.nanmean(i) for i in pocket_distances[run_type]]

# graph
colors = ["#D4A017", "#008080", "#708090", "#691751"] # ! IF YOU WANT TO PUT COLORS, PUT HERE! SAME ORDER AS RUN_TYPE!
fig, ax = plt.subplots(figsize=(20, 8))

titleSize = 32 
labelSize = 25
legendSize = 18

ax.set_title('Average Distance Between FAD and Pocket CoM',fontsize=titleSize)

ax.set_xlabel('Time (ns)',fontsize=labelSize)
ax.set_ylabel('Distance (Å)',fontsize=labelSize)

# Plot the data with the specified colors
for (series, values), color in zip(pocket_distances.items(), colors):
    ax.plot(np.linspace(0, sim_length, len(values)+1)[1:], values, label=series.split("_")[0], color=color)

# Add legend
ax.legend(loc='upper right', shadow=True, ncol=4,fontsize=legendSize)

# Adjust x-ticks for 640 residues, spaced by 50
ax.set_ylim(bottom=6, top=18)

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.style.context("seaborn-talk")

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_SAH_pocket_distance.png', format='png', bbox_inches='tight')  # For SVG format

#### Find Pocket Distance, Each Run Seperate

In [ ]:
# import the VMD distance's found
pocket_distances = {}
for run_type in run_types:
    for run in range(len(run_count)): 
        pocket_distances[f"{run_type.split("_")[0]}_{str(run+1)}"] = []
        with open(f"data_storage/WT_S440W/SAH_pocket_distance/posedistance_{run_type.split("_")[0]}{run+1}.dat", "r") as f:
            text = f.read()
            text = text.split("\n")
            for line in text:
                split_line = line.split("	")
                if len(split_line) > 1:
                    pocket_distances[f"{run_type.split("_")[0]}_{str(run+1)}"].append(float(line.split("	")[1]))

ci = 0
all = True # if all run types on one graph - true, else false
run_type_graph = "" # if just one run type, put the name of it here
fig, ax = plt.subplots(figsize=(20, 8))

titleSize = 32 
labelSize = 25
legendSize = 18

if all:
    colors = ["#F7BD2B", "#C78F01", "#5A440B", "#42DBDB", "#019E9E", "#002929"]
else:
    colors = ["#D4A017", "#008080", "#708090", "#691751"] 
    remove = []
    for key in pocket_distances.keys(): # graph just WT_v11fig, ax = plt.subplots(figsize=(20, 8))
        if not key.startswith(run_type_graph):
            remove.append(key)
    for key in remove:
        del pocket_distances[key]
    

ax.set_title('Distance Between SAH and Pocket CoM',fontsize=titleSize)

ax.set_xlabel('Time (ns)',fontsize=labelSize)
ax.set_ylabel('Distance (Å)',fontsize=labelSize)

# Plot the data with the specified colors
for (series, values), color in zip(pocket_distances.items(), colors):
    ax.plot(np.linspace(0, sim_length, len(values)+1)[1:], values, label=f"{series.split("_")[0]}_{series[-1]}", color=color)

# Add legend
ax.legend(loc='upper left', shadow=True, ncol=4,fontsize=legendSize)

# Adjust x-ticks for 640 residues, spaced by 50
ax.set_ylim(bottom=6, top=18)

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.style.context("seaborn-talk")

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_all_run_FAD_pocket_distance.png', format='png', bbox_inches='tight')  # For SVG format